In [38]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Example") \
    .getOrCreate()

Генерируем простой датасет

In [39]:
import datetime

start_date = datetime.date(2022, 5, 15)
data_set = [(start_date + datetime.timedelta(days=i), 1) for i in range(60)]

Загружаем в датафрейм

In [40]:
df = spark.createDataFrame(data=data_set, schema=["date", "value"])
df.show()

+----------+-----+
|      date|value|
+----------+-----+
|2022-05-15|    1|
|2022-05-16|    1|
|2022-05-17|    1|
|2022-05-18|    1|
|2022-05-19|    1|
|2022-05-20|    1|
|2022-05-21|    1|
|2022-05-22|    1|
|2022-05-23|    1|
|2022-05-24|    1|
|2022-05-25|    1|
|2022-05-26|    1|
|2022-05-27|    1|
|2022-05-28|    1|
|2022-05-29|    1|
|2022-05-30|    1|
|2022-05-31|    1|
|2022-06-01|    1|
|2022-06-02|    1|
|2022-06-03|    1|
+----------+-----+
only showing top 20 rows



Агрегируем по моему варианту

In [41]:
df_agg = df.withColumn("week", F.date_trunc("week", "date")) \
    .withColumn("month", F.date_trunc("month", "date")) \
    .groupBy("month", "week").agg(F.sum("value").alias("sum")) \
    .orderBy("week")

Обрати внимание на границы месяцев

In [42]:
df_agg.show()

+-------------------+-------------------+---+
|              month|               week|sum|
+-------------------+-------------------+---+
|2022-05-01 00:00:00|2022-05-09 00:00:00|  1|
|2022-05-01 00:00:00|2022-05-16 00:00:00|  7|
|2022-05-01 00:00:00|2022-05-23 00:00:00|  7|
|2022-05-01 00:00:00|2022-05-30 00:00:00|  2|
|2022-06-01 00:00:00|2022-05-30 00:00:00|  5|
|2022-06-01 00:00:00|2022-06-06 00:00:00|  7|
|2022-06-01 00:00:00|2022-06-13 00:00:00|  7|
|2022-06-01 00:00:00|2022-06-20 00:00:00|  7|
|2022-06-01 00:00:00|2022-06-27 00:00:00|  4|
|2022-07-01 00:00:00|2022-06-27 00:00:00|  3|
|2022-07-01 00:00:00|2022-07-04 00:00:00|  7|
|2022-07-01 00:00:00|2022-07-11 00:00:00|  3|
+-------------------+-------------------+---+



Тестируем группировку по месяцам

In [43]:
df_agg.groupBy("month").agg(F.sum("sum")).orderBy("month").show()

+-------------------+--------+
|              month|sum(sum)|
+-------------------+--------+
|2022-05-01 00:00:00|      17|
|2022-06-01 00:00:00|      30|
|2022-07-01 00:00:00|      13|
+-------------------+--------+



Тестируем группировку по неделям

In [44]:
df_agg.groupBy("week").agg(F.sum("sum")).orderBy("week").show()

+-------------------+--------+
|               week|sum(sum)|
+-------------------+--------+
|2022-05-09 00:00:00|       1|
|2022-05-16 00:00:00|       7|
|2022-05-23 00:00:00|       7|
|2022-05-30 00:00:00|       7|
|2022-06-06 00:00:00|       7|
|2022-06-13 00:00:00|       7|
|2022-06-20 00:00:00|       7|
|2022-06-27 00:00:00|       7|
|2022-07-04 00:00:00|       7|
|2022-07-11 00:00:00|       3|
+-------------------+--------+

